In [1]:
import numpy as np
import pandas as pd
import string
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
### read train.csv and drop NaN
dataframe = pd.read_csv('dataset/train.csv')
temp = dataframe.dropna()


In [3]:
# ### run test.csv
# dataframe_2 = pd.read_csv('dataset/test.csv')

# ### Run test on 300000 that both train and test set have.
# # print(dataframe_2.shape)
# df_unknown = dataframe.drop(temp.index)
# # print(df_unknown.shape) # 300072
# diff = df_unknown[['Id']].drop(np.array(dataframe_2['Id']))
# df_2 = df_unknown[['Id']].drop(diff.index)
# test_metadata = dataframe.iloc[np.array(df_2).reshape(300000,),:]
# # print(test_metadata.shape) # 300000

(300000, 2)
(300072, 9)
(300000, 9)


In [4]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
             "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
             "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
             "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", 
             "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as",
             "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through",
             "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",
             "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", 
             "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", 
             "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should",
             "now"]

In [5]:
### Preprocess train data
reviews_df2 = temp.sample(n=200)
reviews_df2['helpful%'] = np.round_(np.where(reviews_df2['HelpfulnessDenominator'] > 0,
                                  reviews_df2['HelpfulnessNumerator'] / reviews_df2['HelpfulnessDenominator'], -1),1)

reviews_df2['helpfulness_range'] = pd.cut(x=reviews_df2['helpful%'], bins=[-1, -0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
                                         labels=['0', '1', '2', '3', '4', '5'], include_lowest=True)

# for comments, get rid of punctuation and make them as all lowercase

def text_process(reviewText):
    nopunc = [i for i in reviewText if i not in string.punctuation]
    nopunc = [nopunc.lower() for nopunc in nopunc]
    nopunc_text = ''.join(nopunc)
    return [i for i in nopunc_text.split() if i not in stopwords]
text = []

for i in range(reviews_df2['Summary'].shape[0]):
    if(i%10000==0):
        print(i)
    text.append(text_process(np.array(reviews_df2['Summary'])[i]))
reviews_df2['Summary'] = text

text2 = []
for i in range(reviews_df2['Text'].shape[0]):
    text2.append(text_process(np.array(reviews_df2['Text'])[i]))
reviews_df2['Text'] = text2

0


In [ ]:
### preprocessing test data
# reviews_df = test_metadata


# # create a parameter to check whether a score is helpful, if no one votes helpful at all, certain comment will
# # return -1

# reviews_df['helpful%'] = np.round_(np.where(reviews_df['HelpfulnessDenominator'] > 0,
#                                   reviews_df['HelpfulnessNumerator'] / reviews_df['HelpfulnessDenominator'], -1),1)

# reviews_df['helpfulness_range'] = pd.cut(x=reviews_df['helpful%'], bins=[-1, -0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
#                                          labels=['0', '1', '2', '3', '4', '5'], include_lowest=True)

# # for comments, get rid of punctuation and make them as all lowercase

# def text_process(reviewText):
#     nopunc = [i for i in reviewText if i not in string.punctuation]
#     nopunc = [nopunc.lower() for nopunc in nopunc]
#     nopunc_text = ''.join(nopunc)
#     return [i for i in nopunc_text.split() if i not in stopwords]
# text = []
# for i in range(reviews_df['Summary'].shape[0]):
#     if i%10000==0:
#         print(i)
#     text.append(text_process(np.array(reviews_df['Summary'])[i]))
# reviews_df['Summary'] = text

# text2 = []
# for i in range(reviews_df['Text'].shape[0]):
#     text2.append(text_process(np.array(reviews_df['Text'])[i]))
# reviews_df['Text'] = text2
# print(reviews_df[['Summary','Text']])
# print(reviews_df[['HelpfulnessNumerator','HelpfulnessDenominator','helpful%','helpfulness_range']])

In [9]:
### count score of positive and negative words
# if positive word is found +1, negative word is found -1
with open('negative-words.txt', 'r',encoding = "ISO-8859-1") as f:
    x = f.readlines()
neg_words = np.char.strip(np.array(x),'\n')

with open('positive-words.txt', 'r',encoding = "ISO-8859-1") as f:
    y = f.readlines()
pos_words = np.char.strip(np.array(y),'\n')
print(neg_words.shape,pos_words.shape)

score_list = []
score = 0
# print(reviews_df['Summary'])
for j in range(reviews_df2['Text'].shape[0]):
    list_words = np.array(reviews_df2['Text'])[j]
    for k in range(len(list_words)):
        if list_words[k] in pos_words:
            score = score + 1
        elif list_words[k] in neg_words:
            score = score - 1
    score_list.append(score)
    score = 0
score_list_scaled = preprocessing.scale(score_list)
reviews_df2['Text_score'] = np.round(score_list_scaled,2)

score = 0
score_list = []
for j in range(reviews_df2['Summary'].shape[0]):
    if(j%10000==0):
        print(j)
    list_words = np.array(reviews_df2['Summary'])[j]
    for k in range(len(list_words)):
        if list_words[k] in pos_words:
            score = score + 1
        elif list_words[k] in neg_words:
            score = score - 1
    score_list.append(score)
    score = 0
score_list_scaled = preprocessing.scale(score_list)
reviews_df2['Summary_score'] = np.round(score_list_scaled,2)
print("done")


(4781,) (2005,)
0
done


In [11]:
### Weight text_score and summary_score
alpha = 0
final_score = (reviews_df2['Summary_score']*alpha + reviews_df2['Text_score'])/(1+alpha)
reviews_df2['Final_score'] = np.round(final_score,2)

In [12]:
# split train and label
tr = reviews_df2[['Id','ProductId','UserId',
         'HelpfulnessNumerator','HelpfulnessDenominator']]
# change string to integer
col = reviews_df2['ProductId']
reviews_df2[['Id','ProductId','UserId',
         'HelpfulnessNumerator','HelpfulnessDenominator']] = tr.apply(lambda col: pd.factorize(col)[0])
reviews_df2['Time'] = preprocessing.scale(reviews_df2['Time'])
print(reviews_df2.shape)

# tr = reviews_df2[['Id','ProductId','UserId',
#          'HelpfulnessNumerator','HelpfulnessDenominator']]
# # change string to integer
# col = reviews_df2['ProductId']
# reviews_df2[['Id','ProductId','UserId',
#          'HelpfulnessNumerator','HelpfulnessDenominator']] = tr.apply(lambda col: pd.factorize(col)[0])
# reviews_df2['Time'] = preprocessing.scale(reviews_df2['Time'])

(200, 14)


In [14]:
# test_ = test_metadata[['Id','ProductId','UserId','HelpfulnessNumerator','HelpfulnessDenominator','Time']]
# col_ = test_['ProductId']
# test = test_.apply(lambda col_: pd.factorize(col_)[0]).sample(n=5000)
# df2 = temp.drop(df.index)
# df2 = df2.sample(n=20000)
# # split train and label
# tr2 = df2[['Id','ProductId','UserId','HelpfulnessNumerator','HelpfulnessDenominator']] #,'Time'
# # change string to integer
# col2 = df2['ProductId']
# x_test = tr2.apply(lambda col2: pd.factorize(col2)[0])
# print(x_test)
# y_test = df2[['Score']]
# print(reviews_df)
x = reviews_df2[['ProductId','UserId','HelpfulnessDenominator','HelpfulnessNumerator']]
y = reviews_df2['Score']
# x_test = reviews_df[['ProductId','UserId','HelpfulnessDenominator','HelpfulnessNumerator']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# print(x_train.shape)

In [15]:
print(x_test.shape)

(40, 4)


In [17]:
list_ypred = []
# Logistic Regression

logreg = LogisticRegression(solver='lbfgs',multi_class='multinomial')
logreg.fit(x_train,y_train)
Y_pred = logreg.predict(x_test)
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_log = round(logreg.score(x_train, y_train) * 100, 2)
acc_log
# list_ypred.append(count/Y_pred.shape[0])

0.6


60.62

In [19]:
# Support Vector Machines
svc = LinearSVC()
svc.fit(x_train, y_train)
Y_pred = svc.predict(x_test)
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_svc = round(svc.score(x_train, y_train) * 100, 2)
acc_svc
# print(Y_pred)
# # list_ypred.append(count/Y_pred.shape[0])
# reviews_df['Score'] = Y_pred
# reviews_df = reviews_df[['Id','Score']]
# print(reviews_df.shape)
# reviews_df.to_csv('dataset/output.csv')

0.55


60.0

In [ ]:
#print(y_test, Y_pred[:60,])

In [21]:
# K Nearest Neighbors Classifier

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train, y_train)
Y_pred = knn.predict(x_test)
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_knn = round(knn.score(x_train, y_train) * 100, 2)
acc_knn
# list_ypred.append(count/Y_pred.shape[0])

0.5


64.38

In [22]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
Y_pred = sgd.predict(x_test)
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)
acc_sgd
# list_ypred.append(count/Y_pred.shape[0])

0.55


59.38

In [23]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
Y_pred = decision_tree.predict(x_test)
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree
# list_ypred.append(count/Y_pred.shape[0])

0.4


100.0

In [25]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
Y_pred = random_forest.predict(x_test)
# print(Y_pred)
# print(np.array(y_test).reshape(x_test.shape[0],))
count = 0
for i in range(Y_pred.shape[0]):
    if (Y_pred[i,]-np.array(y_test).reshape(x_test.shape[0],)[i,] == 0):
        count +=1
print(count/Y_pred.shape[0])
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
acc_random_forest
# list_ypred.append(count/Y_pred.shape[0])

0.45


100.0

In [26]:
print(list_ypred)
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest', 
              'Stochastic Gradient Decent','Decision Tree'],
    'Score': list_ypred})
models.sort_values(by='Score', ascending=False)

[0.5]


ValueError: arrays must all be same length